### Importing Libraries

In [3]:
import numpy as np
import matplotlib as plt
import pandas as pd


### Task Outline
We have ODI dataset that contains several columns of different teams. 
From  this dataset we will be separating our columns and creating a new dataframe. which we are caling table. I hope i am not understanding it wrong.
Becasue when we create these  separate tables then we can preprocess it, and then then access varoius columns from these separate tables to get singel out put. 
 Task:
In this we are concerned with batsman performance only.
He can be out in ways like given below so fir "OUT TYPE" column give the follwing values.
- bold out then write: 1
- LB: 2
- Catch: 3
- R.O: 4
- stump:5

2nd table:
- In this table separate the data of pakistan.

3rd table: 
- From the second table, remove the batting team from pakistani data becuase we know its them so just remove where there is pakisatmi team
- remove non striker

4th table:
- keep match id, ball, bowling team, bowler, runs of the bat, out type.

now this onwards we need to do somwork. 
lets take shaihd afridi, and we want his all data. Shahid afridi vs Bowler 1, 2 3 etc., his runs against each bowler, how many times he bold out etc.
Then his score in each over. in 20 games. then we can take his average ...




### Loading the dataset

In [5]:
df=pd.read_csv(r"C:\Users\PMLS\Downloads\ODI_ball_by_ball_updated.csv\ODI_ball_by_ball_updated.csv")
df.head(5)

# runs of the  batter, bowler, out

C:\Users\PMLS\AppData\Local\Temp\ipykernel_25680\1138615557.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r"C:\Users\PMLS\Downloads\ODI_ball_by_ball_updated.csv\ODI_ball_by_ball_updated.csv")


,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,64814,2002,2002-12-29,"McLean Park, Napier",1,0.1,New Zealand,India,SP Fleming,NJ Astle,...,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,64814,2002,2002-12-29,"McLean Park, Napier",1,0.2,New Zealand,India,NJ Astle,SP Fleming,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,64814,2002,2002-12-29,"McLean Park, Napier",1,0.3,New Zealand,India,NJ Astle,SP Fleming,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,64814,2002,2002-12-29,"McLean Park, Napier",1,0.4,New Zealand,India,NJ Astle,SP Fleming,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,64814,2002,2002-12-29,"McLean Park, Napier",1,0.5,New Zealand,India,SP Fleming,NJ Astle,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### First table: Wicket type
- Now our first goal is to assign the numbers to specific wicket condition if its a lbw then number assign to it is 2. Before that in our dataset its a string value. So to correct that we have assigned specfic numbers to eachh string value.
- But before we need to know how many rows in our dataset do actually have a string instead of just "Nan" . becuase not every time a batsman is out , sometimes the over ends or simply the match.
- So what we will do is that where there is a string value we will replace it with a number and where there is Nan, we will replace it default "10" which means the batsman was not out yet .

In [67]:
# Chceking what excatly in wicket type column inorder to map numbers. Printing where the valye is not null
wickets_df = df[df["wicket_type"].notna()]
print(wickets_df[["wicket_type"]].head()) 

num_nan = df["wicket_type"].isna().sum() # storing the sum of count where there  is Nan value
print("Number of NaN values in wicket_type column:", num_nan)

num_non_nan = df["wicket_type"].notna().sum()  # In this variable storing all the count where there is a string vlaue
print("Number of non-NaN values in wicket_type column:", num_non_nan)




    wicket_type
18       caught
203      caught
216         lbw
259      caught
268     run out
Number of NaN values in wicket_type column: 1385611
Number of non-NaN values in wicket_type column: 38657


- Now that we know what is the string value we can assign numbers to each of it.
- Fisrt we created a dictionary of number we want to map and then we used (.map)  function that replaces the string values in df dataframe with corresponding numbers in it.
- We have also assigned a number 10 to those rows of wicket _out column where there is a nan value.
- Then we display that column alone to see if it has wokred or not.

In [69]:

# defining the mapping for 'wicket_type' column
out_type_mapping = {
    "bowled": 1,
    "lbw": 2,
    "caught": 3,
    "run out": 4,
    "stumped": 5
}
df["wicket_type"] = df["wicket_type"].map(out_type_mapping)

df["wicket_type"].fillna(10, inplace=True) # replacing the nan value is 10 and applying the same chabge to the dataset by inplace = true

print(df[["wicket_type"]].head())

# Now the df dataframe has the same modified column of wicket type.


   wicket_type
0         10.0
1         10.0
2         10.0
3         10.0
4         10.0


C:\Users\PMLS\AppData\Local\Temp\ipykernel_17192\2555977630.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["wicket_type"].fillna(10, inplace=True) # replacing the nan value is 10 and applying the same chabge to the dataset by inplace = true


### 2nd Table

- Now we want to have only pakistani team data so from the dataframe  we will extractthe column of batting team attribute and will save it a separate variable.
- we hav eapplied a condition == pakistan. This will check condition on each row and if condition matched the it will extract that rows data to tis -- - variable other wise it will skip that.

In [71]:
pakistan_df = df[df["batting_team"] == "Pakistan"]

pakistan_df.head()


,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
9580,65238,2002,2003-02-11,"New Wanderers Stadium, Johannesburg",2,0.1,Pakistan,Australia,Taufeeq Umar,Shahid Afridi,...,0,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN
9581,65238,2002,2003-02-11,"New Wanderers Stadium, Johannesburg",2,0.2,Pakistan,Australia,Taufeeq Umar,Shahid Afridi,...,0,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN
9582,65238,2002,2003-02-11,"New Wanderers Stadium, Johannesburg",2,0.3,Pakistan,Australia,Taufeeq Umar,Shahid Afridi,...,1,1.0,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN
9583,65238,2002,2003-02-11,"New Wanderers Stadium, Johannesburg",2,0.4,Pakistan,Australia,Taufeeq Umar,Shahid Afridi,...,0,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN
9584,65238,2002,2003-02-11,"New Wanderers Stadium, Johannesburg",2,0.5,Pakistan,Australia,Taufeeq Umar,Shahid Afridi,...,0,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN


### 3rd Table


- now in the new dataframe " pkaistan _df" we only have data of pakistani team  so obviously batting team will be pakistani and on the non string end there will also be a pakistani batter since its their batting.So, for making our datset concise and correct we will remkove these or drop these 2 columns from our new datframe.

In [73]:
pakistan_df = pakistan_df.drop(columns=["batting_team", "non_striker"])

pakistan_df.head()


,match_id,season,start_date,venue,innings,ball,bowling_team,striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
9580,65238,2002,2003-02-11,"New Wanderers Stadium, Johannesburg",2,0.1,Australia,Taufeeq Umar,GD McGrath,0,0,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN
9581,65238,2002,2003-02-11,"New Wanderers Stadium, Johannesburg",2,0.2,Australia,Taufeeq Umar,GD McGrath,0,0,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN
9582,65238,2002,2003-02-11,"New Wanderers Stadium, Johannesburg",2,0.3,Australia,Taufeeq Umar,GD McGrath,0,1,1.0,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN
9583,65238,2002,2003-02-11,"New Wanderers Stadium, Johannesburg",2,0.4,Australia,Taufeeq Umar,GD McGrath,0,0,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN
9584,65238,2002,2003-02-11,"New Wanderers Stadium, Johannesburg",2,0.5,Australia,Taufeeq Umar,GD McGrath,0,0,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN


- Now from teh 3rd tabke we will further drop unrelated column that we dont need and will keep the necessary columns only .
- The columns whichh are present in the dataframe are the most important one for further analysis with respect to pakistani team only so, the rest of the columns will be dropped.

In [104]:
pakistan_filtered_df = pakistan_df[["match_id", "ball", "bowling_team","striker","bowler", "runs_off_bat", "wicket_type"]]

pakistan_filtered_df.head(20)


,match_id,ball,bowling_team,striker,bowler,runs_off_bat,wicket_type
9580,65238,0.1,Australia,Taufeeq Umar,GD McGrath,0,10.0
9581,65238,0.2,Australia,Taufeeq Umar,GD McGrath,0,10.0
9582,65238,0.3,Australia,Taufeeq Umar,GD McGrath,0,10.0
9583,65238,0.4,Australia,Taufeeq Umar,GD McGrath,0,10.0
9584,65238,0.5,Australia,Taufeeq Umar,GD McGrath,0,10.0
9585,65238,0.6,Australia,Taufeeq Umar,GD McGrath,0,10.0
9586,65238,0.7,Australia,Taufeeq Umar,GD McGrath,0,10.0
9587,65238,1.1,Australia,Shahid Afridi,JN Gillespie,0,10.0
9588,65238,1.2,Australia,Shahid Afridi,JN Gillespie,0,10.0
9589,65238,1.3,Australia,Shahid Afridi,JN Gillespie,0,10.0


### Shahid Afridi data 

- afridi_df = pakistan_filtered_df[pakistan_filtered_df["striker"] == "Shahid Afridi"]
- afridi_df.head(30)


##### Creating a separate menu for him.

.loc :
for this we will use our grouped data from afridi_matches variable which looks like this:
index match id  bowling team   runs_off bat
01      9776      newzealand       89
02    34677       bangladesh      124  etc   
now it will search this and will return the index number through .loc for example it will return 2 and give us something like:
runs off bat =124
match id= 34677
bowling team = bangladesh 


In [118]:
# creating a separate dataframe only for shahid afridi where only his data will be ppresent.

afridi_df = df[df["striker"] == "Shahid Afridi"]
df["runs_off_bat"] = pd.to_numeric(df["runs_off_bat"], errors="coerce") # making sure that all the runs in the dataframe are numeric 

afridi_matches = afridi_df.groupby(["match_id", "bowling_team"])["runs_off_bat"].sum().reset_index() 

"""in this case we are grouping each row where match id is same + bowling team is same , it means that its the same match and we
are summing up runs he made on each ball he made in same match, this will be later used in if else loop for highest runs made in match, minimum etc"""

def afridi_stats():  
    while True:
        print("\n📊 Shahid Afridi's Stats Menu")
        print("1. Total matches played")
        print("2. Total runs in ODIs")
        print("3. Highest runs in a match")
        print("4. Average runs in ODIs")
        print("5. Matches where he got out on 0")
        print("6. Dismissal Stats (How he got out)")
        print("7. Exit")
        
 #taking input from user.
        choice = input("Enter your choice (1-7): ")

        if choice == "1":
            total_matches = afridi_df["match_id"].nunique()  # it will count the match ids that are unique,  and will print it.
            print(f"🏏 Total matches played: {total_matches}")
            print(" =================================================")

        elif choice == "2":
            total_runs = afridi_df["runs_off_bat"].sum() # above we made sure that this column has only numbers, now we will sum up every value of this column since it belongs to shahid's only so we will have his entire ODI's runs
            print(f"🏏 Total runs scored: {total_runs}")
            print(" =================================================")
            
        elif choice == "3":
            highest_match = afridi_matches.loc[afridi_matches["runs_off_bat"].idxmax()]
            print(f"🔥 Highest score: {highest_match['runs_off_bat']} vs {highest_match['bowling_team']} (Match ID: {highest_match['match_id']})")
            print(" =================================================")
            
        elif choice == "4":
            avg_runs = afridi_matches["runs_off_bat"].mean()   # it will sum up all the runsof the matches and will return the average of it
            print(f"📊 Average runs in ODIs: {avg_runs:.2f}")
            print(" =================================================")


        elif choice == "5":
            duck_matches = afridi_matches[afridi_matches["runs_off_bat"] == 0] # this checks the afridi dataframe and store only thee rows  where runs off bat ==0 in new duck matches dataframe
            
            if duck_matches.empty:
                print("✅ Afridi never got out on 0 in this dataset!")  
                print(" =================================================")
            else:
                print("🦆 Matches where Afridi got out on 0:")
                print(duck_matches[['match_id', 'bowling_team']].to_string(index=False)) #to string extracts only the match_id and bowling_team columns from duck_matches.
                print(" =================================================")  #index=false converts the daataFrame into a readable string without row numbers (index).

        elif choice == "6":
            print("\n🚨 How Shahid Afridi Got Out 🚨")
            print(f" - Caught: {afridi_df['wicket_type'].eq(3).sum()} times")
            print(f" - Bowled: {afridi_df['wicket_type'].eq(1).sum()} times")
            print(f" - LBW: {afridi_df['wicket_type'].eq(2).sum()} times")
            print(f" - Run Out: {afridi_df['wicket_type'].eq(4).sum()} times")
            print(f" - Stumped: {afridi_df['wicket_type'].eq(5).sum()} times")
            print(f" - Not Out: {afridi_df['wicket_type'].eq(10).sum()} times")
            print(" =================================================")

            
        elif choice == "7":   # letting the user exit 
            print("👋 Exiting... Goodbye!")
            print(" =================================================")
            break

        else:
            print("❌ Invalid choice! Please enter a number between 1-7.")
            print(" =================================================")

# Run the menu
afridi_stats()



📊 Shahid Afridi's Stats Menu
1. Total matches played
2. Total runs in ODIs
3. Highest runs in a match
4. Average runs in ODIs
5. Matches where he got out on 0
6. Dismissal Stats (How he got out)
7. Exit


Enter your choice (1-7):  6



🚨 How Shahid Afridi Got Out 🚨
 - Caught: 116 times
 - Bowled: 26 times
 - LBW: 9 times
 - Run Out: 7 times
 - Stumped: 6 times
 - Not Out: 2834 times

📊 Shahid Afridi's Stats Menu
1. Total matches played
2. Total runs in ODIs
3. Highest runs in a match
4. Average runs in ODIs
5. Matches where he got out on 0
6. Dismissal Stats (How he got out)
7. Exit


Enter your choice (1-7):  7


👋 Exiting... Goodbye!
